<a href="https://colab.research.google.com/github/Grundmane/Google_playstore_data_end_to_end_project/blob/main/Google_playstore_data_end_to_end_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Google playstore data

AGENDA

We have google playstore dataset containing info of different apps ionstalled rating, versions and other details and we are going to do analysis based on the data we have

1) find out 10 top reviews given to the apps

2) Top 100 installs apps and distribution of type free-paid

3) Category wise distribution of installed apps

4) Top paid apps


In [ ]:
!pip install pyspark

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import *

In [ ]:
spark = SparkSession.builder.appName('PlayStoreProject').getOrCreate()

Create data frame

In [ ]:
from html import escape
df = spark.read.load('googlestore.xlsx - Sheet1.csv',format='csv',header=True,inferSchema=True)

In [ ]:
df.count()

10841

In [ ]:
df.show(1)

+--------------------+--------------+------+-------+--------+----+-----+------------+
|                 App|      Category|Rating|Reviews|Installs|Type|Price|      Genres|
+--------------------+--------------+------+-------+--------+----+-----+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159|   10000|Free|    0|Art & Design|
+--------------------+--------------+------+-------+--------+----+-----+------------+
only showing top 1 row



Check schema

In [ ]:
df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)



data cleaning step

In [ ]:
df=df.drop("Size","Content Rating","Last Updated","Android Ver","Current Ver")

In [ ]:
df.show(3)

+--------------------+--------------+------+-------+----------+----+-----+--------------------+
|                 App|      Category|Rating|Reviews|  Installs|Type|Price|              Genres|
+--------------------+--------------+------+-------+----------+----+-----+--------------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159|   10,000+|Free|    0|        Art & Design|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967|  500,000+|Free|    0|Art & Design;Pret...|
|U Launcher Lite â...|ART_AND_DESIGN|   4.7|  87510|5,000,000+|Free|    0|        Art & Design|
+--------------------+--------------+------+-------+----------+----+-----+--------------------+
only showing top 3 rows



In [ ]:
df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Genres: string (nullable = true)



In [ ]:
from pyspark.sql.functions import regexp_replace, col

In [ ]:
df=df.withColumn("Reviews", col("Reviews").cast(IntegerType()))\
.withColumn("Installs", regexp_replace(col("Installs"), "[^0-9]", ""))\
.withColumn("Installs",col("Installs").cast(IntegerType()))\
.withColumn("Price", regexp_replace(col("Price"), "[$]",""))\
.withColumn("Price",col("Price").cast(IntegerType()))

In [ ]:
df.show(3)

+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|                 App|      Category|Rating|Reviews|Installs|Type|Price|              Genres|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159|   10000|Free|    0|        Art & Design|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967|  500000|Free|    0|Art & Design;Pret...|
|U Launcher Lite â...|ART_AND_DESIGN|   4.7|  87510| 5000000|Free|    0|        Art & Design|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
only showing top 3 rows



In [ ]:
df.createOrReplaceTempView("PlayStoreProject")

In [ ]:
PlayStoreSelect = spark.sql("SELECT * FROM PlayStoreProject")
PlayStoreSelect.show()

+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|                 App|      Category|Rating|Reviews|Installs|Type|Price|              Genres|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159|   10000|Free|    0|        Art & Design|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967|  500000|Free|    0|Art & Design;Pret...|
|U Launcher Lite â...|ART_AND_DESIGN|   4.7|  87510| 5000000|Free|    0|        Art & Design|
|Sketch - Draw & P...|ART_AND_DESIGN|   4.5| 215644|50000000|Free|    0|        Art & Design|
|Pixel Draw - Numb...|ART_AND_DESIGN|   4.3|    967|  100000|Free|    0|Art & Design;Crea...|
|Paper flowers ins...|ART_AND_DESIGN|   4.4|    167|   50000|Free|    0|        Art & Design|
|Smoke Effect Phot...|ART_AND_DESIGN|   3.8|    178|   50000|Free|    0|        Art & Design|
|    Infinite Painter|ART_AND_DESIGN|   4.1|  36815| 1000000

1) top reviews given to the apps

In [ ]:
top_reviews = spark.sql("select App,sum(Reviews) from PlayStoreProject group by 1 order by 2 desc")

In [ ]:
top_reviews.show(10)

+--------------------+------------+
|                 App|sum(Reviews)|
+--------------------+------------+
|           Instagram|   266241989|
|  WhatsApp Messenger|   207348304|
|      Clash of Clans|   179558781|
|Messenger â€“ Tex...|   169932272|
|      Subway Surfers|   166331958|
|    Candy Crush Saga|   156993136|
|            Facebook|   156286514|
|         8 Ball Pool|    99386198|
|        Clash Royale|    92530298|
|            Snapchat|    68045010|
+--------------------+------------+
only showing top 10 rows



Top 10 installed

In [ ]:
top_installed = spark.sql("select App,Type,sum(Installs) from PlayStoreProject group by 1,2 order by 3 desc")
top_installed.show(10)

+--------------------+----+-------------+
|                 App|Type|sum(Installs)|
+--------------------+----+-------------+
|      Subway Surfers|Free|   6000000000|
|           Instagram|Free|   4000000000|
|        Google Drive|Free|   4000000000|
|            Hangouts|Free|   4000000000|
|       Google Photos|Free|   4000000000|
|         Google News|Free|   4000000000|
|    Candy Crush Saga|Free|   3500000000|
|Messenger â€“ Tex...|Free|   3000000000|
|  WhatsApp Messenger|Free|   3000000000|
|               Gmail|Free|   3000000000|
+--------------------+----+-------------+
only showing top 10 rows



Category wise distribution of installed apps

In [ ]:
category_dist_installed = spark.sql("select Category,sum(Installs) from PlayStoreProject group by 1 order by 2 desc")
category_dist_installed.show()

+-------------------+-------------+
|           Category|sum(Installs)|
+-------------------+-------------+
|               GAME|  35086024415|
|      COMMUNICATION|  32647276251|
|       PRODUCTIVITY|  14176091369|
|             SOCIAL|  14069867902|
|              TOOLS|  11452771915|
|             FAMILY|  10258263505|
|        PHOTOGRAPHY|  10088247655|
| NEWS_AND_MAGAZINES|   7496317760|
|   TRAVEL_AND_LOCAL|   6868887146|
|      VIDEO_PLAYERS|   6222002720|
|           SHOPPING|   3247848785|
|      ENTERTAINMENT|   2869160000|
|    PERSONALIZATION|   2325494782|
|BOOKS_AND_REFERENCE|   1921469576|
|             SPORTS|   1751174498|
| HEALTH_AND_FITNESS|   1582072512|
|           BUSINESS|   1001914865|
|            FINANCE|    876648734|
|          EDUCATION|    871452000|
|MAPS_AND_NAVIGATION|    719281890|
+-------------------+-------------+
only showing top 20 rows



Top paid apps

In [ ]:
top_paid_app = spark.sql("SELECT App, SUM(Installs) AS TotalInstalls FROM PlayStoreProject WHERE Type='Paid' GROUP BY 1 ORDER BY TotalInstalls DESC")
top_paid_app.show(10)

+--------------------+-------------+
|                 App|TotalInstalls|
+--------------------+-------------+
|           Minecraft|     20000000|
|       Hitman Sniper|     10000000|
| Facetune - For Free|      3000000|
|Beautiful Widgets...|      2000000|
|          HD Widgets|      2000000|
|   Cut the Rope GOLD|      1000000|
|          True Skate|      1000000|
|League of Stickma...|      1000000|
|Infinity Dungeon VIP|      1000000|
|Five Nights at Fr...|      1000000|
+--------------------+-------------+
only showing top 10 rows



Top paid rating apps